In [1]:
import numpy as np
import pandas as pd

In [2]:
with open('./eval.txt', 'r') as f:
    results = f.readlines()

# Remove any non-result lines from the eval file, and split the lines on the tab character
# (results have format: model_name\tdataset_name\tmetric_name\tmetric_value)
results = [r.replace('\n','').split('\t') for r in results if '\t' in r]

In [3]:
def remove_underscore_after(val): # Remove underscore, and keep the part after the underscore
    return val.split('_')[-1]
def remove_underscore_before(val): # Remove underscore, and keep the part before the underscore
    return val.split('_')[0]
def map2d(func, grid): # Mapping for 2d arrays, from: https://stackoverflow.com/questions/70742445/elegant-map-over-2d-list
    return [[func(value) for value in row] for row in grid]

df = pd.DataFrame(results, columns = ['model', 'dataset', 'metric', 'value'])
df['value'] = pd.to_numeric(df['value'])
df['model'] = df['model'].map(lambda x: '-'.join(x.split('-')[2:]))
models = df['model'].str.split('-').tolist()

# Remove all underscores from our 2d list, keep one list of the param names and one with param vals
model_names_list = map2d(remove_underscore_before, models)
model_names = np.array(model_names_list)
model_val_list = map2d(remove_underscore_after, models)
model_vals = np.array(model_val_list)

In [4]:
model_vals.shape, model_names.shape

((440, 13), (440, 13))

In [5]:
# Splitting model name into more columns
for i in range(model_names.shape[-1]):
    name = model_names[0][i]
    val = model_vals[:,i]
    df[name] = val
    try:
       df[name] = pd.to_numeric(df[name]) 
    except:
        pass

# Remove the column that specifies which data fold we used
df = df.drop(['fold', 'AL', 'PL', 'vit', 'model', 'data', 'ALL', 'method', 'ratio', 'kw'], axis = 1)

cols = sorted(df.columns.tolist())
# The ratio have the format: 'ratio_xxx' where xxx is some float.
# We only want that float value, and need to convert it from string to float.
# df['ratio'] = pd.to_numeric(df['ratio'])
display(df)

# Group by all columns except the performance values, then compute mean, std and number of model runs for the performance
df_grouped = df.groupby(list(set(cols)-set(['value']))).agg({'value':['mean', 'std', 'count']})

,dataset,metric,value,epochs,lr,bs
0,RSICD-CLS,zeroshot-val-top1,3.47,10,0.00050,64
1,UCM-CLS,zeroshot-val-top1,5.71,10,0.00050,64
2,RSICD,image_to_text_R@1,0.09,10,0.00050,64
3,RSICD,image_to_text_R@5,0.46,10,0.00050,64
4,RSICD,image_to_text_R@10,0.91,10,0.00050,64
...,...,...,...,...,...,...
435,Sydney,image_to_text_R@5,10.66,15,0.00005,64
436,Sydney,image_to_text_R@10,19.52,15,0.00005,64
437,Sydney,text_to_image_R@1,4.02,15,0.00005,64
438,Sydney,text_to_image_R@5,13.68,15,0.00005,64


In [6]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(df_grouped)

value                
                                                      mean       std count
metric             epochs lr       bs dataset                             
image_to_text_R@1  10     0.000005 64 RSICD       3.127500  0.454927     4
                                      Sydney      2.062500  0.503678     4
                                      UCM         9.645000  0.983887     4
                          0.000050 64 RSICD       4.592500  0.399114     4
                                      Sydney      2.617500  0.405000     4
                                      UCM        10.832500  2.788768     4
                          0.000500 64 RSICD       0.105000  0.036742     6
                                      Sydney      0.266667  0.163299     6
                                      UCM         0.558333  0.191877     6
                   15     0.000050 64 RSICD       5.440000  0.338132     4
                                      Sydney      3.370000  0.965091     4
                                      UCM        10.237500  1.480053     4
                          0.000500 64 RSICD       0.090000  0.073485     4
                                      Sydney      0.200000  0.163299     4
                                      UCM         0.955000  0.548483     4
image_to_text_R@10 10     0.000005 64 RSICD      23.585000  1.228563     4
                                      Sydney     14.385000  0.991985     4
                                      UCM        62.022500  2.178323     4
                          0.000050 64 RSICD      30.622500  1.930878     4
                                      Sydney     18.412500  1.596901     4
                                      UCM        73.215000  4.704696     4
                          0.000500 64 RSICD       0.805000  0.174671     6
                                      Sydney      2.311667  0.645923     6
                                      UCM         5.633333  1.716411     6
                   15     0.000050 64 RSICD      32.152500  1.193828     4
                                      Sydney     19.567500  2.118763     4
                                      UCM        79.047500  1.027663     4
                          0.000500 64 RSICD       1.577500  1.466183     4
                                      Sydney      2.715000  2.288879     4
                                      UCM         7.497500  4.862334     4
image_to_text_R@5  10     0.000005 64 RSICD      13.230000  0.956278     4
                                      Sydney      8.602500  1.186462     4
                                      UCM        40.360000  1.571814     4
                          0.000050 64 RSICD      18.122500  1.416295     4
                                      Sydney     10.615000  1.202816     4
                                      UCM        40.955000  3.035891     4
                          0.000500 64 RSICD       0.490000  0.092952     6
                                      Sydney      1.210000  0.334664     6
                                      UCM         2.938333  0.702692     6
                   15     0.000050 64 RSICD      19.262500  0.700589     4
                                      Sydney     11.467500  1.469725     4
                                      UCM        46.430000  2.110624     4
                          0.000500 64 RSICD       0.845000  0.844768     4
                                      Sydney      1.460000  1.337934     4
                                      UCM         3.810000  2.860000     4
text_to_image_R@1  10     0.000005 64 RSICD       2.287500  0.513314     4
                                      Sydney      2.767500  0.758304     4
                                      UCM         9.047500  1.027663     4
                          0.000050 64 RSICD       4.502500  0.392290     4
                                      Sydney      3.320000  0.346410     4
                                      UCM        10.597500  2.033591     4
   

In [7]:
df_grouped.to_csv('eval.csv')

In [8]:
df_2 = df_grouped.reset_index()
# df_2 = df_2[(df_2['method'] == 'ours') & (df_2['ratio'] == 0.2) & (df_2['PL'] == 'ot.image')] # (df_2['metric'] == 'image_to_text_R@5') &
df_2 = df_2[(df_2['epochs'] > 5)  & (df_2['bs'] == 64)] # &(df_2['lr']==5e-5)
df_2 = df_2.sort_values(['metric', 'dataset', ('value', 'mean')])
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(df_2)

metric epochs        lr  bs    dataset      value            \
                                                             mean       std   
12   image_to_text_R@1     15  0.000500  64      RSICD   0.090000  0.073485   
6    image_to_text_R@1     10  0.000500  64      RSICD   0.105000  0.036742   
0    image_to_text_R@1     10  0.000005  64      RSICD   3.127500  0.454927   
3    image_to_text_R@1     10  0.000050  64      RSICD   4.592500  0.399114   
9    image_to_text_R@1     15  0.000050  64      RSICD   5.440000  0.338132   
13   image_to_text_R@1     15  0.000500  64     Sydney   0.200000  0.163299   
7    image_to_text_R@1     10  0.000500  64     Sydney   0.266667  0.163299   
1    image_to_text_R@1     10  0.000005  64     Sydney   2.062500  0.503678   
4    image_to_text_R@1     10  0.000050  64     Sydney   2.617500  0.405000   
10   image_to_text_R@1     15  0.000050  64     Sydney   3.370000  0.965091   
8    image_to_text_R@1     10  0.000500  64        UCM   0.558333  0.191877   
14   image_to_text_R@1     15  0.000500  64        UCM   0.955000  0.548483   
2    image_to_text_R@1     10  0.000005  64        UCM   9.645000  0.983887   
11   image_to_text_R@1     15  0.000050  64        UCM  10.237500  1.480053   
5    image_to_text_R@1     10  0.000050  64        UCM  10.832500  2.788768   
21  image_to_text_R@10     10  0.000500  64      RSICD   0.805000  0.174671   
27  image_to_text_R@10     15  0.000500  64      RSICD   1.577500  1.466183   
15  image_to_text_R@10     10  0.000005  64      RSICD  23.585000  1.228563   
18  image_to_text_R@10     10  0.000050  64      RSICD  30.622500  1.930878   
24  image_to_text_R@10     15  0.000050  64      RSICD  32.152500  1.193828   
22  image_to_text_R@10     10  0.000500  64     Sydney   2.311667  0.645923   
28  image_to_text_R@10     15  0.000500  64     Sydney   2.715000  2.288879   
16  image_to_text_R@10     10  0.000005  64     Sydney  14.385000  0.991985   
19  image_to_text_R@10     10  0.000050  64     Sydney  18.412500  1.596901   
25  image_to_text_R@10     15  0.000050  64     Sydney  19.567500  2.118763   
23  image_to_text_R@10     10  0.000500  64        UCM   5.633333  1.716411   
29  image_to_text_R@10     15  0.000500  64        UCM   7.497500  4.862334   
17  image_to_text_R@10     10  0.000005  64        UCM  62.022500  2.178323   
20  image_to_text_R@10     10  0.000050  64        UCM  73.215000  4.704696   
26  image_to_text_R@10     15  0.000050  64        UCM  79.047500  1.027663   
36   image_to_text_R@5     10  0.000500  64      RSICD   0.490000  0.092952   
42   image_to_text_R@5     15  0.000500  64      RSICD   0.845000  0.844768   
30   image_to_text_R@5     10  0.000005  64      RSICD  13.230000  0.956278   
33   image_to_text_R@5     10  0.000050  64      RSICD  18.122500  1.416295   
39   image_to_text_R@5     15  0.000050  64      RSICD  19.262500  0.700589   
37   image_to_text_R@5     10  0.000500  64     Sydney   1.210000  0.334664   
43   image_to_text_R@5     15  0.000500  64     Sydney   1.460000  1.337934   
31   image_to_text_R@5     10  0.000005  64     Sydney   8.602500  1.186462   
34   image_to_text_R@5     10  0.000050  64     Sydney  10.615000  1.202816   
40   image_to_text_R@5     15  0.000050  64     Sydney  11.467500  1.469725   
38   image_to_text_R@5     10  0.000500  64        UCM   2.938333  0.702692   
44   image_to_text_R@5     15  0.000500  64        UCM   3.810000  2.860000   
32   image_to_text_R@5     10  0.000005  64        UCM  40.360000  1.571814   
35   image_to_text_R@5     10  0.000050  64        UCM  40.955000  3.035891   
41   image_to_text_R@5     15  0.000050  64        UCM  46.430000  2.110624   
57   text_to_image_R@1     15  0.000500  64      RSICD   0.067500  0.045000   
51   text_to_image_R@1     10  0.000500  64      RSICD   0.090000  0.000000   
45   text_to_image_R@1     10  0.000005  64      RSICD   2.287500  0.513314   
48   text_to_image_R@1     10  0.000050  64      RSICD   4.502500 